In [52]:
#############################################################################
#                    用来将重复经纬度的数据进行归并                               #
#                    最后把数据划分成 经度 维度 数据重复项 重复数据                 #
#                                  LON LAT  N        [N1,N2,……Nn]           #
#                    最后写在Excel并且输出                                     #
#############################################################################
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from openpyxl import load_workbook,Workbook

In [59]:

'''
    1.首先在Excel中求得经纬度的平均值，列标签要求为LON，LAT
    #虽然这一步在代码中可以实现，但是这边求懒了
'''


def write(ExcelPath,SavePath,Savepig,ThresholdValue=100):

    '''
    :param ExcelPath: 原始数据的绝对地址
    :param SavePath: 存放生成数据的Excel绝对地址
    :param Savepig: 存储图片的地址
    :param ThresholdValue:阈值，默认=100
    :return:None
    '''

    df1 = pd.read_excel(ExcelPath)
    df1 = pd.DataFrame(df1)

    N = df1[['LAT','LON','M']]
    gb = N.groupby(['LAT','LON'])
    #BaseData会统计经纬度的概率数量
    BaseData = pd.DataFrame(gb.count())
    BaseData.to_excel(SavePath)
    '''
 
        2.
        基础经纬度和重复项已经写进去Data.xlsx中
        接下来把重复项的具体项写进去
        使用{(LAT,LON)]:[M1,M2,……,Mn]}
    '''
    #将数据使用字典绑定，key是元组(LAT,LON)，Value是[M1,M2,……,Mn]
    dict = {}
    for i in range(len(N)):
        if dict.get((N['LAT'][i],N['LON'][i])) == None:  # 在字典中查找key，因为创建的是空字典，每个key第一次get结果都是None
            dict[(N['LAT'][i],N['LON'][i])] = []
            dict[(N['LAT'][i],N['LON'][i])].append(N['M'][i])
        else:
            dict[(N['LAT'][i], N['LON'][i])].append(N['M'][i])
    #对dict排序，从而得到和BaseData一样的排序
    index = sorted(dict)
    '''
    3.
    直方图
    纵坐标是数量：
    横坐标是概率：
    '''
    #阈值
    ThresholdValue = 100
    ##先读入文件路径
    writer = pd.ExcelWriter(SavePath, engine='openpyxl',mode='a')
    writer.book = load_workbook(SavePath)

    ws = writer.book["Sheet1"]  # 指定工作表
    #len(dict)+2=975
    for i in range(2,len(dict)+2):
        if ws.cell(row=i, column=3).value >= ThresholdValue:
            print(ws.cell(row=i, column=3).value)
            #hist的参数可以根据需要修改
            n, bins, patches = plt.hist(dict[(index[i - 2][0],index[i - 2][1])],bins=50)
            plt.xlabel("M")
            plt.ylabel("Amount")
            plt.title(str(index[i - 2][0])+","+ str(index[i - 2][1]))
            fig = plt.gcf()
            fig.savefig(Savepig+str(i)+".png")
            plt.close()
        for j in range(len(dict[(index[i-2][0],index[i-2][1])])):
            ws.cell(row=i, column=j+4).value = dict[(index[i-2][0],index[i-2][1])][j]
    writer.save()  # 保存后才能成功改动
    writer.close()


In [60]:

write('C:\\Users\\lenovo\Desktop\\prediction_results\\Majorprediction.xlsx','C:\\Users\\lenovo\Desktop\\prediction_results\\Data.xlsx','D:\\Python\\histogram\\',100)

230
128
104
162
102
325
124
183
101
129
100
124
109
388
138
105
103
208
304
125
139
192
162
142
149
262
117
281
1051
141
532
105
115
283
107
131
256
530
122
256
116
